In addition to the other sources (Uniprot, official Website), there is another Uniprot annotation file from EBI. Are there any differences? Is the EBI file better than the other options?

According to the comments in the file, it is generated by Uniprot, and contains annotations for all proteins in Uniprot! This file could contain the qualifiers and evidence codes we have been missing from the Uniprot custom download file. 

## TODO:

- Analyze file
- Filter down
    - Molecular function
    - enables
- Compare to other GO datasets
    - GO annotation from Uniprot custom download
    - Uniprot annotation from GO website
        - IEA, non-IEA
- Write script for filtering file, and saving it as xz

In [1]:
# !This file contains all GO annotations and gene product information for proteins in the UniProt KnowledgeBase (UniProtKB),
# !ComplexPortal protein complexes, and RNAcentral identifiers.
# !
# !date-generated: 2022-11-17 13:42
# !generated-by: UniProt
# !go-version: http://purl.obolibrary.org/obo/go/releases/2022-11-12/extensions/go-plus.owl
# !


In [2]:
import pandas as pd

go_official_path="/home/ad/gene_ontology/goa_uniprot_all_iea.tsv.xz"
go_ebi_path="/home/ad/ebi_goa/goa_uniprot_all_iea.tsv.xz"


TODO remove drop_dupl, sort_values after command has finished

In [3]:
from subpred.util import pickle_read

In [5]:
df_go_official = pickle_read(
    go_official_path,
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
)

reading /home/ad/gene_ontology/goa_uniprot_all_iea.tsv.xz.1675436275.0136392.pkl


In [6]:
df_go_ebi = pickle_read(
    go_ebi_path,
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
).drop_duplicates()


cache file /home/ad/ebi_goa/goa_uniprot_all_iea.tsv.xz.1675783942.2537284.pkl did not exist, creating new cache:


In [7]:
concatenated = pd.concat([df_go_ebi, df_go_official])

In [8]:
diff = concatenated.drop_duplicates(keep=False)

In [9]:
diff2 = diff.drop("date", axis=1)


In [10]:
diff2_nodup = diff2.drop_duplicates(ignore_index=True)

In [11]:
diff2_nodup

,Uniprot,qualifier,go_id,evidence_code,aspect
0,A0A000,enables,GO:0003824,IEA,F
1,A0A000,enables,GO:0003870,IEA,F
2,A0A000,enables,GO:0030170,IEA,F
3,A0A000,involved_in,GO:0009058,IEA,P
4,A0A000,involved_in,GO:0033014,IEA,P
...,...,...,...,...,...
732652181,Z9JY54,enables,GO:0046872,IEA,F
732652182,Z9JY54,enables,GO:0051539,IEA,F
732652183,Z9JZ13,enables,GO:0016407,IEA,F
732652184,Z9JZ30,involved_in,GO:0050790,IEA,P


In [12]:
diff2 = diff.drop("date", axis=1)

In [13]:
diff

,Uniprot,qualifier,go_id,evidence_code,aspect,date
0,A0A000,enables,GO:0003824,IEA,F,20221109
1,A0A000,enables,GO:0003870,IEA,F,20221109
2,A0A000,enables,GO:0030170,IEA,F,20221109
3,A0A000,involved_in,GO:0009058,IEA,P,20221109
4,A0A000,involved_in,GO:0033014,IEA,P,20221109
...,...,...,...,...,...,...
703804717,Z9JZ82,involved_in,GO:0046677,IEA,P,20220907
703804718,Z9JZ82,involved_in,GO:0055085,IEA,P,20220907
703804719,Z9JZ82,located_in,GO:0005886,IEA,C,20220907
703804720,Z9JZ82,located_in,GO:0016020,IEA,C,20220907


In [14]:
diff.date.value_counts()

20221109    689139394
20220907    679019154
20220909     24500110
20221111     23774159
20170228       775493
              ...    
20051119            1
20010328            1
20171014            1
20030713            1
20220501            1
Name: date, Length: 7080, dtype: int64

TODO maybe the date is different?

In [15]:
from subpred.go_utils import GeneOntology, GO_FILE

go_owl = GeneOntology(GO_FILE)
